In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
import string
import re
import collections
from sklearn import preprocessing

%matplotlib inline

In [2]:
# READ DATA 
train_df = pd.read_json('train.json.zip')
test_df = pd.read_json('test.json.zip')

## TRAIN DATA FEATURE ENGINEERING

In [3]:
# convert TARGET to the numeric
train_df['interest_level'] = train_df['interest_level'].apply(lambda x: 0 if x=='low' 
                                                      else 1 if x=='medium' 
                                                      else 2) 
# REMOVE UNNECESSARY WORDS FROM DESCRIPTION
train_df['description'] = train_df['description'].apply(lambda x: x.replace("<br />", ""))
train_df['description'] = train_df['description'].apply(lambda x: x.replace("br", ""))
train_df['description'] = train_df['description'].apply(lambda x: x.replace("<p><a", ""))

#basic features
train_df['rooms'] = train_df['bedrooms'] + train_df['bathrooms'] 

# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))

# description contains email
regex = r'[\w\.-]+@[\w\.-]+'
train_df['has_email'] = train_df['description'].apply(lambda x: 1 if re.findall(regex, x) else 0)

# description contains phone
train_df['has_phone'] = train_df['description'].apply(lambda x:re.sub('['+string.punctuation+']', '', x).split())\
        .apply(lambda x: [s for s in x if s.isdigit()])\
        .apply(lambda x: len([s for s in x if len(str(s))==10]))\
        .apply(lambda x: 1 if x>0 else 0)

# CONVERT LOWER ALL OF WORDS
train_df[["features"]] = train_df[["features"]].apply(
    lambda _: [list(map(str.strip, map(str.lower, x))) for x in _])

## TEST DATA FEATURE ENGINEERING

In [4]:
# REMOVE UNNECESSARY WORDS FROM DESCRIPTION
test_df['description'] = test_df['description'].apply(lambda x: x.replace("<br />", ""))
test_df['description'] = test_df['description'].apply(lambda x: x.replace("br", ""))
test_df['description'] = test_df['description'].apply(lambda x: x.replace("<p><a", ""))

#basic features
test_df['rooms'] = test_df['bedrooms'] + test_df['bathrooms'] 

# count of photos #
test_df["num_photos"] = test_df["photos"].apply(len)

# count of "features" #
test_df["num_features"] = test_df["features"].apply(len)

# count of words present in description column #
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

# description contains email
regex = r'[\w\.-]+@[\w\.-]+'
test_df['has_email'] = test_df['description'].apply(lambda x: 1 if re.findall(regex, x) else 0)

# description contains phone
test_df['has_phone'] = test_df['description'].apply(lambda x:re.sub('['+string.punctuation+']', '', x).split())\
        .apply(lambda x: [s for s in x if s.isdigit()])\
        .apply(lambda x: len([s for s in x if len(str(s))==10]))\
        .apply(lambda x: 1 if x>0 else 0)

# CONVERT LOWER ALL OF WORDS
test_df[["features"]] = test_df[["features"]].apply(
    lambda _: [list(map(str.strip, map(str.lower, x))) for x in _])

## MOST FREQUENT FEATURES EXTRACTION

In [5]:
feature_value_train = train_df['features'].tolist()
feature_value_test = test_df['features'].tolist()

feature_value_train
feature_value_test

feature_lst_train = []
feature_lst_test = []

for i in range(len(feature_value_train)):
    feature_lst_train += feature_value_train[i]
    
for i in range(len(feature_value_test)):
    feature_lst_test += feature_value_test[i]

uniq_feature_train = list(set(feature_lst_train))
uniq_feature_test = list(set(feature_lst_test))


# see the frequency of each feature
def most_common(lst):
    features = collections.Counter(lst)
    feature_value = features.keys()
    frequency = features.values()
    data = [('feature_value', feature_value),
            ('frequency', frequency),]    
    df = pd.DataFrame.from_dict(dict(data))
    return df.sort_values(by = 'frequency', ascending = False)

df_features_train = most_common(feature_lst_train)
df_features_test = most_common(feature_lst_test)


def newColumn(name, df, series):
    feature = pd.Series(0,df.index,name = name)# data : 0
    for row,word in enumerate(series):
        if name in word:
            feature.iloc[row] = 1
    df[name] = feature # feature : series ; value in series : 1 or 0
    return df

# select features based on frequency
facilities = ['elevator', 'cats allowed', 'hardwood floors', 'dogs allowed', 'doorman', 'dishwasher', 'no fee', 'laundry in building', 'fitness center']
for name in facilities:
    train_df = newColumn(name, train_df, train_df['features'])
    test_df = newColumn(name, test_df, test_df['features'])

## LABEL ENCODING

In [6]:
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))

In [7]:
train_df['price'] = np.log10(train_df['price'])
test_df['price'] = np.log10(test_df['price'])

In [8]:
train_df['price']

4         3.380211
6         3.579784
9         3.543447
10        3.477121
15        3.446382
            ...   
124000    3.447158
124002    3.379306
124004    3.267172
124008    3.622732
124009    3.631444
Name: price, Length: 49352, dtype: float64

## DROP UNNECESSARY COLUMNS

In [9]:
# TRAINING DATASET
train_df.drop('created', axis=1, inplace=True)
train_df.drop('description', axis=1, inplace=True)
train_df.drop('features', axis=1, inplace=True)
train_df.drop('photos', axis=1, inplace=True)


# TEST DATASET
test_df.drop('created', axis=1, inplace=True)
test_df.drop('description', axis=1, inplace=True)
test_df.drop('features', axis=1, inplace=True)
test_df.drop('photos', axis=1, inplace=True)

# REGRESSION FOR PRICE

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import optuna
import math
from keras import callbacks

X = train_df.drop(['price'], axis = 1)
y = train_df.price
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = .3,
                                                    random_state = 5)

In [11]:
class Optimizer:
    def __init__(self, metric, trials=100):
        self.metric = metric
        self.trials = trials
        
    def objective(self, trial):
        model = create_model(trial)
        model.fit(X, y)
        preds = model.predict(X_test)
        return mean_absolute_error(y_test, preds)
            
    def optimize(self):
        study = optuna.create_study(direction="minimize")
        study.optimize(self.objective, n_trials=self.trials)
        return study

In [12]:
from keras.layers import Dense
from keras.models import Sequential

def create_model(trial):
    model = Sequential()
    model.add(Dense
                 (
                    units = trial.suggest_int("units", 32, 256),
                    kernel_initializer=trial.suggest_categorical("kernel_initializer", ["glorot_uniform", "normal"]),
                    input_dim = X.shape[1], 
                    activation= trial.suggest_categorical("activation", ["relu", "silu", "leaky_relu"])
                 )
             )
    model.add(Dense(1))
    model.compile(
        loss="mean_absolute_error"
    )
    return model

optimizer = Optimizer('mae')
keras_study = optimizer.optimize()

print("Number of finished trials: ", len(keras_study.trials))
print("Best trial:")
keras_trial = keras_study.best_trial

print("  Value: {}".format(keras_trial.value))
print("  Params: ")
for key, value in keras_trial.params.items():
    print("    {}: {}".format(key, value))


[I 2021-09-28 17:53:36,049] A new study created in memory with name: no-name-a331b82c-4720-4590-9c92-fed25c0a8156


1543/1543 [==============================] - 2s 889us/step - loss: 27321.9727


[I 2021-09-28 17:53:39,674] Trial 0 finished with value: 16515.011623708084 and parameters: {'units': 177, 'kernel_initializer': 'normal', 'activation': 'silu'}. Best is trial 0 with value: 16515.011623708084.


1543/1543 [==============================] - 2s 781us/step - loss: 54704.8359


[I 2021-09-28 17:53:41,675] Trial 1 finished with value: 90180.31850626413 and parameters: {'units': 203, 'kernel_initializer': 'glorot_uniform', 'activation': 'relu'}. Best is trial 0 with value: 16515.011623708084.


...

1543/1543 [==============================] - 2s 744us/step - loss: 17194.4453


[I 2021-09-28 17:56:55,723] Trial 98 finished with value: 14620.5028092472 and parameters: {'units': 41, 'kernel_initializer': 'normal', 'activation': 'relu'}. Best is trial 76 with value: 844.9858726442183.


1543/1543 [==============================] - 2s 799us/step - loss: 20250.0156


[I 2021-09-28 17:56:57,790] Trial 99 finished with value: 20931.296469822406 and parameters: {'units': 49, 'kernel_initializer': 'normal', 'activation': 'silu'}. Best is trial 76 with value: 844.9858726442183.


Number of finished trials:  100
Best trial:
  Value: 844.9858726442183
  Params: 
    units: 42
    kernel_initializer: normal
    activation: relu


In [13]:
keras_params = keras_study.best_params

model2 = Sequential()
model2.add(Dense(**keras_params))
model2.add(Dense(**keras_params))
model2.add(Dense(**keras_params))
model2.add(Dense(1, activation= keras_params['activation']))

model2.compile(loss="mean_absolute_error")
earlystopping = callbacks.EarlyStopping(monitor ="loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)
model2.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=100, batch_size=10, callbacks =[earlystopping])


Epoch 1/100
3455/3455 [==============================] - 4s 1ms/step - loss: 9.4946 - val_loss: 3.5171
Epoch 2/100
3455/3455 [==============================] - 4s 1ms/step - loss: 3.5175 - val_loss: 3.5171
Epoch 3/100
3455/3455 [==============================] - 4s 1ms/step - loss: 3.5175 - val_loss: 3.5171
Epoch 4/100
3455/3455 [==============================] - 4s 1ms/step - loss: 3.5175 - val_loss: 3.5171
Epoch 5/100
3455/3455 [==============================] - 4s 1ms/step - loss: 3.5175 - val_loss: 3.5171
Epoch 6/100
3455/3455 [==============================] - 3s 1ms/step - loss: 3.5175 - val_loss: 3.5171
Epoch 7/100
3455/3455 [==============================] - 4s 1ms/step - loss: 3.5175 - val_loss: 3.5171
Epoch 8/100
3455/3455 [==============================] - 4s 1ms/step - loss: 3.5175 - val_loss: 3.5171
Epoch 9/100
3455/3455 [==============================] - 4s 1ms/step - loss: 3.5175 - val_loss: 3.5171
Epoch 10/100
3455/3455 [==============================] - 4s 1ms/step - l

In [14]:
print(model2.get_config())

{'name': 'sequential_100', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 25), 'dtype': 'float64', 'sparse': False, 'ragged': False, 'name': 'dense_200_input'}}, {'class_name': 'Dense', 'config': {'name': 'dense_200', 'trainable': True, 'dtype': 'float32', 'units': 42, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'RandomNormal', 'config': {'mean': 0.0, 'stddev': 0.05, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}}, {'class_name': 'Dense', 'config': {'name': 'dense_201', 'trainable': True, 'dtype': 'float32', 'units': 42, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'RandomNormal', 'config': {'mean': 0.0, 'stddev': 0.05, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_re

* BEFORE PREDICTION INVERSE LOG10

In [15]:
train_df['price'] = 10 ** train_df['price']
test_df['price'] = 10 ** test_df['price']

In [16]:
train_df['price']

4         2400.0
6         3800.0
9         3495.0
10        3000.0
15        2795.0
           ...  
124000    2800.0
124002    2395.0
124004    1850.0
124008    4195.0
124009    4280.0
Name: price, Length: 49352, dtype: float64

In [17]:
y_test = 10 ** y_test

In [18]:
score = model2.evaluate(X_test, y_test, verbose=0)
print(score)

3963.275390625
